In [22]:
import os
import subprocess
import sys
import numpy as np
import nrrd

In [2]:
prog_dir = r'W:\People\Yannan\Release'  ### location of binary

aff_cmd = os.path.join(prog_dir, "cat-aff.exe") ## Affine registration command
print(aff_cmd)

bsp_cmd = os.path.join(prog_dir, "cat-bsp.exe") ## non-rigid Bspline registration command
print(bsp_cmd)

rfm_cmd = os.path.join(prog_dir, "cat-rfm.exe") ## for Reformatting, to apply alignment parameters to other datasets
print(rfm_cmd)



W:\People\Yannan\Release\cat-aff.exe
W:\People\Yannan\Release\cat-bsp.exe
W:\People\Yannan\Release\cat-rfm.exe


In [3]:

def run_code(fixed_name, fixed_fn, dat_dir, out_dir, useMask="1", start_match_str = '', end_match_str = '.nrrd', reformat=True, aff=False, bsp=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_fn_Aff = os.path.join(out_dir,  rfm_str + '_Rfm_Aff.nrrd')

                aff_param_fn = os.path.join(out_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(out_dir, rfm_str + '_BspP.txt')

                aff_log_fn = os.path.join(out_dir, rfm_str + '_Aff_log.txt')
                aff_log_fn_err = os.path.join(out_dir, rfm_str + '_Aff_log_err.txt')

                bsp_log_fn = os.path.join(out_dir, rfm_str + "_Bsp_log.txt")
                bsp_log_fn_err = os.path.join(out_dir, rfm_str + "_Bsp_log_err.txt")

                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")


                with open(aff_log_fn,'w+') as fout:
                    with open(aff_log_fn_err,'w+') as ferr:
                        out=subprocess.call([aff_cmd, fixed_fn, moving_fn, aff_param_fn], stdout=fout, stderr=ferr)
                if (reformat and aff):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn_Aff, "0", "1"], stdout=fout,stderr=ferr)
                if(bsp):
                    with open(bsp_log_fn,'w+') as fout:
                        with open(bsp_log_fn_err,'w+') as ferr:
                            out=subprocess.call([bsp_cmd, fixed_fn, moving_fn, aff_param_fn, bsp_param_fn, useMask], stdout=fout,stderr=ferr)
                    if (reformat):
                        with open(rfm_log_fn,'w+') as fout:
                            with open(rfm_log_fn_err,'w+') as ferr:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)

                        
def align_chn(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_iD(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True, str_Add='_10cum'):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + str_Add
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
                    
def align_chn_iD_autoF(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True, str_Add='_10cum'):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + str_Add
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_488autofluor", "_647")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_iD_autoF488(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True, str_Add='_10cum'):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + str_Add
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_488", "_647")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_iD_noMod(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True, str_Add='_10cum'):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + str_Add
                rfm_fn = os.path.join(out_dir,  fixed_name + '_' + fn + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
#                 rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)    
                    
def align_chn_iD_apply(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', ref_name = '',\
                       bsp_transform=True, str_Add='_10cum'):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                

                    
                rfm_fn = os.path.join(out_dir,  fixed_name + '_' + fn + '_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, fixed_name + '_' + fn + str_Add + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, fixed_name + '_' + fn + str_Add + "_Rfm_log_err.txt")
#                 rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, fixed_name + '_' + ref_name + str_Add + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, fixed_name + '_' + ref_name + str_Add + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_C_highRes(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + '_10um'
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_fn_aff = os.path.join(out_dir,  rfm_str + '_Rfm_Aff.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn_aff, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_C_highRes_647(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn + '_10um'
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_fn_aff = os.path.join(out_dir,  rfm_str + '_Rfm_Aff.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_561", "_647")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn_aff, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_C(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn 
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_fn_aff = os.path.join(out_dir,  rfm_str + '_Rfm_Aff.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn_aff, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_C_647(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = fixed_name + '_' + fn 
                rfm_fn = os.path.join(out_dir,  rfm_str + '_Rfm.nrrd')
                rfm_fn_aff = os.path.join(out_dir,  rfm_str + '_Rfm_Aff.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
                rfm_str = rfm_str.replace("_561", "_647")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "1"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn_aff, "0", "1"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)
                    
def align_chn_ABA_annot(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = '', end_match_str = '.nrrd', bsp_transform=True):
    for root2, dirs2, files2 in os.walk(dat_dir):
        for fn in files2:
            if (fn.endswith(end_match_str)) and (fn.startswith(start_match_str)):
                print(fn)
                moving_fn = os.path.join(dat_dir,fn) #this needs to be swapped if not aligning with Allen
                fn = fn.rsplit(".", 1)[0]
                rfm_str = 'iDAvg_4Ims_R2_10cum_ccfv3_17_average_template_25_VD_20pt5-crp'
                rfm_fn = os.path.join(out_dir,  'ABA_annot_Rfm.nrrd')
                rfm_log_fn = os.path.join(out_dir, rfm_str + "_Rfm_log.txt")
                rfm_log_fn_err = os.path.join(out_dir, rfm_str + "_Rfm_log_err.txt")
#                 rfm_str = rfm_str.replace("_647", "_561")
                aff_param_fn = os.path.join(param_dir, rfm_str + '_AffP.txt')
                bsp_param_fn = os.path.join(param_dir, rfm_str + '_BspP.txt')

                
                if (os.path.exists(aff_param_fn)):
                    with open(rfm_log_fn,'w+') as fout:
                        with open(rfm_log_fn_err,'w+') as ferr:
                            if (bsp_transform):
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "1", "0"], stdout=fout,stderr=ferr)
                            else:
                                out=subprocess.call([rfm_cmd, aff_param_fn, bsp_param_fn, fixed_fn, moving_fn, rfm_fn, "0", "0"], stdout=fout,stderr=ferr)
                else:
                    print("Not Found: " + aff_param_fn)


In [ ]:
data, _ = nrrd.read(r'E:\Yannan\Registration\iDAvg_7Ims_10cum_noOLF_Stitched_skip-400-1400_flip_rot-75_10um_Rfm.nrrd')


In [14]:
## Registration with 10 um data, can be any resolution
rfm_cmd = os.path.join(prog_dir, "cat-rfm.exe")
print(rfm_cmd)

dat_dir = r'Y:\People\Yannan\Segmentation\pLSM_vasculature\bigstitched\nrrd'
out_dir = r'E:\Yannan\Registration'
fixed_name = 'iDAvg_7Ims_10cum_noOLF'
fixed_fn = os.path.join(r'Y:\People\Yannan\Segmentation\pLSM_vasculature\ABA', fixed_name + '.nrrd')
run_code(fixed_name, fixed_fn, dat_dir, out_dir, useMask="0", start_match_str = 'Stitched_skip-400-1400_flip_rot-75_10um', end_match_str = '.nrrd', reformat=True, aff=True)


W:\People\Yannan\Release\cat-rfm.exe
Stitched_skip-400-1400_flip_rot-75_10um.nrrd


In [4]:
## Applying registration parameter to transform high-res images. cat-rfm-big_iD442x has hard-coded parameters 
#to eventually tranform to 2.5, 2.5, 5 resolution

rfm_cmd = os.path.join(prog_dir, 'Reformat', "cat-rfm-big_iD442x.exe")
print(rfm_cmd)

dat_dir = r'Y:\People\Yannan\Segmentation\pLSM_vasculature\bigstitched\nrrd'
param_dir = r'E:\Yannan\Registration'
out_dir = r'E:\Yannan\Registration'

fixed_name = 'iDAvg_7Ims_10cum_noOLF'
fixed_fn = os.path.join(r'Y:\People\Yannan\Segmentation\pLSM_vasculature\ABA', fixed_name + '.nrrd')

align_chn_iD_noMod(fixed_name, fixed_fn, dat_dir, param_dir, out_dir, start_match_str = 'Stitched_skip-400-1400_flip', \
                   end_match_str = '75.nrrd', bsp_transform=True, str_Add='_10um')

rfm_cmd = os.path.join(prog_dir, "cat-rfm.exe")
print(rfm_cmd)



W:\People\Yannan\Release\Reformat\cat-rfm-big_iD442x.exe
Stitched_skip-400-1400_flip_rot-75.nrrd
W:\People\Yannan\Release\cat-rfm.exe
